# Default dataframe

In [14]:
import pandas as pd
import PyPDF2
import re
import json
import numpy as np

import os

from PyPDF2 import PdfFileReader
import lxml.etree as ET

In [15]:
os.getcwd()

'c:\\Users\\giaco\\source\\repos\\Thesis\\Data'

In [16]:
# !!! CHANGE DEPENDING ON THE OS !!!
isLinux = False

# Utility functions and constants
default_linux_path = os.getcwd() + "/work/"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded\\")
default_path = default_linux_path if isLinux else default_windows_path
DEFAULT_SAVE_DIR = default_path.replace("\\Downloaded", "\\Generated")

def extract_text_from_pdf(pdf_path, skip_pages=0):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(skip_pages, len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def extract_text_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    return ET.tostring(root, encoding='unicode', method='text')

def extract_text_from_rtf(rtf_path):
    return pypandoc.convert_file(rtf_path, 'plain', format='rtf')

def write_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read()

def split_text(text, pattern):
    parts = re.split(pattern, text, flags=re.MULTILINE)
    
    parts = [part for part in parts if part]
    
    if not re.match(pattern, parts[0]):
        parts = parts[1:]
    
    return parts

## Extraction From Local Files

### Diritto Amministrativo PDF

In [17]:
# Extract questions from the PDF with quiz
DIRAMM_PC_PATH = default_path + "03_diritto_amministrativo.pdf"
DIRAMM_REMOTE_PATH = default_path + "documents/03_diritto_amministrativo.pdf"

def extract_questions_answers(text):
    qa_pattern = re.compile(r'(\d+)\. (.+?)\n(A\) .+?)\n(B\) .+?)\n(C\) .+?)\n', re.DOTALL)
    matches = qa_pattern.findall(text)
    
    data = []
    for match in matches:
        question_number, question, answer_a, answer_b, answer_c = match
        
        # Remove newline characters and clean up answers
        question = question.replace('\n', ' ')
        answer_a = answer_a.replace('\n', ' ').replace('A) ', '')
        answer_b = answer_b.replace('\n', ' ').replace('B) ', '')
        answer_c = answer_c.replace('\n', ' ').replace('C) ', '')
        
        data.append([question, answer_a, answer_b, answer_c])
    
    return data

text = extract_text_from_pdf(DIRAMM_PC_PATH)
qa_data = extract_questions_answers(text)

df_diramm = pd.DataFrame(qa_data, columns=['Question', 'Answer 1', 'Answer 2', 'Answer 3'])

print(df_diramm.shape)
df_diramm.head()

(379, 4)


,Question,Answer 1,Answer 2,Answer 3
0,"Dal punto di vista degli effetti, le autorizza...",Consentono ad un soggetto di non adempiere ad ...,Permettono di esercitare facoltà preesistenti.,Accertano l'esistenza dei presupposti richiest...
1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
2,Con riferimento al riesame con esito demolitor...,Quanto a competenza a disporla spetta all'orga...,Non comparta in nessun caso l'obbligo di provv...,Quanto a competenza a disporla non spetta in n...
3,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...


### JSON di Domande sul C.P

In [18]:
# Extract questions from the JSON file
QUESTIONS_PC_PATH = default_path + "domande.json"
QUESTIONS_REMOTE_PATH = default_path + "documents/domande.json"

with open(QUESTIONS_PC_PATH, 'r') as file:
    data = json.load(file)

questions = []
answers1 = []
answers2 = []
answers3 = []

# Iterate over each question object in the JSON data
for item in data:
    question = item['question']
    answers = item['answers']
    
    # Extract each answer and its correctness
    correct_answer = None
    incorrect_answers = []
    for answer in answers:
        if answer['right']:
            correct_answer = answer['answer']
        else:
            incorrect_answers.append(answer['answer'])
    
    answers1.append(correct_answer)
    answers2.append(incorrect_answers[0])
    answers3.append(incorrect_answers[1])
    
    questions.append(question)

df_domjson = pd.DataFrame({
    'Question': questions,
    'Answer 1': answers1,
    'Answer 2': answers2,
    'Answer 3': answers3
})

df_domjson.to_csv(DEFAULT_SAVE_DIR + 'quiz_json.csv', index=False)

print(df_domjson.shape)
df_domjson.head()

(1740, 4)


,Question,Answer 1,Answer 2,Answer 3
0,Ai sensi dell'art. 240 c.p. Ã¨ sempre ordinata...,dei beni che costituiscono il profitto del rea...,dei servizi e degli strumenti informatici o do...,delle cose che costituiscono il movente del reato
1,"Ai sensi dell'art. 266 c.p. Ã¨ punito, se il f...",fa a militari l'apologia di fatti contrari al ...,istiga gli incaricati di pubblico servizio a v...,istiga i funzionari pubblici a disobbedire ai ...
2,La pena della multa ex art. 24 c.p. consiste n...,non inferiore a euro 50 nÃ© superiore a euro 5...,non inferiore a euro 10 nÃ© superiore a euro 5...,non inferiore a euro 25 nÃ© superiore a euro 2...
3,Ai sensi dell'art. 7 c.p. Ã¨ punito secondo la...,Delitti commessi da pubblici ufficiali a servi...,Delitti contro la personalitÃ del Presidente ...,Delitti di contraffazione della marchiatura de...
4,Tra le pene accessorie per i delitti ex art. 1...,la decadenza o la sospensione dall'esercizio d...,l'estinzione del rapporto di impiego subordina...,l'inabilitazione dagli uffici direttivi delle ...


### INPS PDFs

In [19]:
# List of files to process
quiz_files = ["INPSquiz1.pdf", "INPSquiz2.pdf", "INPSquiz4.pdf", "INPSquiz5.pdf", "INPSquiz6.pdf"]

all_quizzes = []
for filename in quiz_files:
    file_path = default_path + filename
    
    # Extract, clean and parse the text
    text = extract_text_from_pdf(file_path, 1)
    text = re.sub(r'PAGINA \d+ DI \d+', '', text)
    text = re.sub(r'\s+', ' ', text)

    pattern = r"(\d{4})\. (.*?)\s+A\) (.*?)\s+B\) (.*?)\s+C\) (.*?)\s+D\) (.*?)(?=\d{4}\.|$)"
    quizzes = re.findall(pattern, text, re.DOTALL)

    # Append the quizzes to the list
    all_quizzes.extend(quizzes)

# Convert the list of quizzes into a DataFrame
quiz_list = [{'Question': q[1], 'Answer 1': q[2], 'Answer 2': q[3], 'Answer 3': q[4]} for q in all_quizzes]
df_inps = pd.DataFrame(quiz_list)

# Display the DataFrame and optionally save it to a CSV file
print(df_inps.shape)
df_inps.head(20)

(1411, 4)


,Question,Answer 1,Answer 2,Answer 3
0,Ai fini del Codice dell'amministrazione digita...,Certificati elettronici.,Autenticazioni informatiche.,Documenti informatici.
1,Cosa dispone il D.P.R. n. 68/20 05 in merito a...,Il gestore utilizzato dal mittente fornisce al...,La ricevuta di avvenuta consegna non può conte...,La ricevuta di avvenuta consegna è rilasciata ...
2,Nei ricorsi agli organi di giustizia amministr...,I termini processuali previsti sono ridotti al...,"Tutti i termini processuali, compresi quelli p...","Valgono i termini processuali ordinari, salvo ..."
3,A seguto dell'affermazione dell'autonomia gest...,Non può revocare provvedimenti o atti di compe...,Può riservare a sé provvedimenti o atti di com...,Può avocare a sé provvedimenti o atti di compe...
4,In materia di documentazione amministra tiva u...,Deve essere sottoscritta dall'interessato; può...,Deve essere sottoscritta dall'interessato; può...,Deve essere sottoscritta dall'interessato; non...
5,"Quale, tra le seguenti, è un a caratteristica ...",É un rimedio di carattere eccezionale previsto...,É un ricorso ordinario impugnatorio che può es...,Non può essere proposto ad organi statali avve...
6,L'oggetto dell'atto amministrativo....,"É la ""res"" su cui l'atto amministrativo incide.","Può anche non essere lecito, ma deve essere de...",Può essere solo un bene che riguardi l'attivit...
7,I controlli amministrativi che intervengono su...,Preventivi antecedenti.,Successivi.,Di merito.
8,L'obbligo di conclusione esplicita del procedi...,Sia quando il procedimento consegua obbligator...,Solo quando il procedimento consegua ad istanza.,Solo quando il procedimento debba essere inizi...
9,Quale effetto produce la violaz ione di dispos...,Non può comportare la costituzione di rapporti...,Il provvedimento è sottoposto all'esame di un ...,Le amministrazioni hanno l'obbligo di recupera...


### Mininterno JSON (still generating via GPT)

In [20]:
# Extract questions from the PDF with quiz
MININ_PC_PATH = default_path + "quiz_mininterno.json"
MININ_REMOTE_PATH = default_path + "documents/quiz_mininterno.json"

with open(MININ_PC_PATH, 'r') as file:
    data = json.load(file)

data = data["questions"]
print(data)

questions = []
answers1 = []
answers2 = []
answers3 = []

for item in data:
    questions.append(item['question'])
    answers1.append(item['options']["A"])
    answers2.append(item['options']["B"])
    answers3.append(item['options']["C"])

df_minin = pd.DataFrame({
    'Question': questions,
    'Answer 1': answers1,
    'Answer 2': answers2,
    'Answer 3': answers3
})

df_minin.to_csv(DEFAULT_SAVE_DIR + 'quiz_json.csv', index=False)

print(df_minin.shape)
df_minin.head()

[{'number': 1, 'question': "Quale delle indicazioni che seguono Ã¨ conforme alle prescrizioni dell'art. 81 della Costituzione in materia di bilancio dello Stato?", 'options': {'A': 'Con la legge di approvazione del bilancio, sono necessariamente stabiliti nuovi tributi e nuove spese', 'B': "Dopo l'approvazione del bilancio, non Ã¨ possibile approvare leggi che importino nuove e maggiori spese", 'C': "Dopo l'approvazione del bilancio, ogni altra legge che importi nuove e maggiori spese deve indicare i mezzi per farvi fronte", 'D': "Solo prima dell'approvazione del bilancio, ogni altra legge che importi nuove e maggiori spese deve indicare i mezzi per farvi fronte"}}, {'number': 2, 'question': 'Il referendum popolare abrogativo non Ã¨ ammesso... (art. 75 della Costituzione)', 'options': {'A': 'Per le leggi tributarie e di bilancio', 'B': 'Solo per le leggi tributarie', 'C': 'In tema di legge di stabilitÃ\xa0 e di bilancio', 'D': 'Solo in tema di bilancio'}}, {'number': 3, 'question': 'Le

,Question,Answer 1,Answer 2,Answer 3
0,Quale delle indicazioni che seguono Ã¨ conform...,"Con la legge di approvazione del bilancio, son...","Dopo l'approvazione del bilancio, non Ã¨ possi...","Dopo l'approvazione del bilancio, ogni altra l..."
1,Il referendum popolare abrogativo non Ã¨ ammes...,Per le leggi tributarie e di bilancio,Solo per le leggi tributarie,In tema di legge di stabilitÃ e di bilancio
2,"Le Camere approvano annualmente, a mente della...",I bilanci e il rendiconto consuntivo,Nuovi tributi e nuove spese,Solo il rendiconto consuntivo
3,"Le leggi che importano nuove o maggiori spese,...",Devono trovare copertura in sede di legge di s...,Non devono indicare i mezzi per farvi fronte,Devono indicare i mezzi per farvi fronte
4,"(art. 227 e ss., D.Lgs. 267/2000) La redazione...",Ãˆ obbligatoria per tutti gli enti indicati ne...,Ãˆ obbligatoria solo per gli enti con popolazi...,Non Ã¨ mai obbligatoria


# Manual Adds

### From formazionegiuridica.org

In [21]:
#d = {"Question": "", "Answer 1": "", "Answer 2": "", "Answer 3": ""}
d1 = {"Question": "Il danno non patrimoniale", "Answer 1": "È categoria che si estende anche al campo della responsabilità contrattuale", "Answer 2": "È categoria comprendente il c.d. danno biologico e il c.d. danno morale, oltre a quanto specificamente disposto dall'art. 2059 c.c.", "Answer 3": "È risarcibile in conseguenza della commissione di un reato, ai sensi dell'art. 2059 c.c."}
d2 = {"Question": "È possibile esperire giudizio volto a ottenere una pronuncia di condanna all'adempimento del contratto, dopo che si sia precedentemente promosso un giudizio volto, invece, a ottenere una pronuncia giudiziale di risoluzione del contratto?", "Answer 1": "Si qualora la domanda di risoluzione venga rigettata", "Answer 2": "Si ma solo se il primo giudizio si sia risolto con la sua estinzione, senza cioè pronuncia nel merito", "Answer 3": "No, l'art. 1453 c.c. è norma che non tollera deroga alcuna"}
d3 = {"Question": "La transazione del condebito ex art. 1304 c.c.", "Answer 1": "Si riferisce alla transazione che abbia a oggetto l'intero debito", "Answer 2": "Si riferisce alla transazione che abbia a oggetto la quota del debitore con cui è stipulata", "Answer 3": "Non è valida in ossequio al principio per cui il contratto produce effetto solo tra le parti"}
d4 = {"Question": "In che rapporto sono tra di loro secondo la più recente giurisprudenza i reati di cui agli artt. 316 ter e 640 bis c.p.?", "Answer 1": "concorso apparente di norme", "Answer 2": "concorso formale di reati", "Answer 3": "concorso materiale"}
d5 = {"Question": "Quali rapporti intercorrono tra la fattispecie ex art. 316 ter c.p. e i reati di falso", "Answer 1": "la prima fattispecie assorbe la seconda", "Answer 2": "la seconda fattispecie assorbe la prima", "Answer 3": "reato complesso: la seconda fattispecie diviene circostanza aggravante della prima"}

df_forgiur = pd.DataFrame([d1, d2, d3, d4, d5])
df_forgiur.shape

(5, 4)

### From simulatore.simone.it

In [22]:
#d = {"Question": "", "Answer 1": "", "Answer 2": "", "Answer 3": ""}
d = {"Question": "Quando si procede a esecuzione forzata in virtù di un titolo esecutivo costituito da scrittura privata autenticata, il precetto è indispensabile (art. 474 c.p.c.)?", "Answer 1": "Si, e deve contenere la trascrizione integrale della scrittura autenticata.", "Answer 2": "Si, se lo richiede l'obbligato.", "Answer 3": "No, non lo è."}
d1 = {"Question": "In materia di Reddito di cittadinzanza, ai sensi dell'art. 2, comma 1 quarter, D.L. 4/2019, con riferimento ai requisiti patrimoniali, e con specifico riferimento ai beni detenuti all'estero, l'INPS provvede a definire un piano di verifica dei requisiti patrimoniali dichiarati nella dichiarazione sostitutiva:", "Answer 1": "Annualmente, entro il 31 Marzo.", "Answer 2": "Annualmente, entro il 23 Febbraio.", "Answer 3": "Annualmente, entro il 31 Dicembre."}
d2 = {"Question": "L'esecuzione forzata per consegna puà aver luogo in vità di (art. 474 c.p.c):", "Answer 1": "Sentenza.", "Answer 2": "Scrittura privata.", "Answer 3": "Cambiale"}
d3 = {"Question": "Ai sensi dell'art. 71 della Costituizione, a chi appartiene l'iniziativa delle leggi?", "Answer 1": "Al Governo, a ciascun membro delle Camere ed agli organi ed enti ai quali sia conferita da legge costituzionale", "Answer 2": "Esclusivamente al Governo", "Answer 3": "Solo alle camere"}
d4 = {"Question": "Ai sensi dell'art. 71 della Costituizione il popolo esercita l'iniziativa delle leggi?", "Answer 1": "Si, mediante la proposta, da parte di almeno cinquantamila elettori, di un progetto redatto in articoli", "Answer 2": "Si, mediante la proposta, da parte di almeno centomila elettori, di un progetto redatto in articoli", "Answer 3": "No"}
d5 = {"Question": "Secondo l'art. 2 comma 9 della L. 241/90 e ss.mm.ii, la tardiva emanazione di un provvedimento amministrativo da parte di un funzionario incaricato, costituisce anche un elemento di valutazione della performance individuale?", "Answer 1": "Si", "Answer 2": "No, solo una responsabilità di natura disciplinare", "Answer 3": "No, solo una responsibilità di natura amministravito-contabile"}
d6 = {"Question": "Ai sensi dell'art. 18 D.Lgs. 81/2008, rientra tra gli obblighi del datore di lavoro:", "Answer 1": "Fornire ai lavoratori i necessari e idonei dispositivi di protezione individuale, sentito il responsabile del servizio di prevenzione e protezione e il medico competente, ove presente.", "Answer 2": "Verificare affinchè soltanto i lavoratori che hanno riscevuto adeguate istruzioni accedano alle zone che li espongono ad un rischio grave e specifico", "Answer 3": "Richiedere l'osservanza delle misure per il controllo delle situazioni di rischio in caso di emergenza e dare istruzioni affinchè i lavoratori, in caso di pericolo grave, immediato e inevitabile, abbandonino il posto di lavoro o la zona perifoclosa."}
d7 = {"Question": "Ai sensi dell'art. 20 D.Lgs. 81/2015, in quale delle seguenti ipotesi non è ammessa l'apposizione di un termine alla durata di un contratto di lavoro subordinato?:", "Answer 1": "Da parte di datori di lavoro che non hanno effettuato la valutazione dei rischi in applicazione della normativa di tutela della salute e della sicurezza dei lavoratori.", "Answer 2": "Qualora non vi sia il consenso del lavoratore", "Answer 3": "Qualora non vi sia stata autorizzazione dell'ispettorato Nazionale del Lavoro"}
d8 = {"Question": "Chi è il responsabile , secondo l'art. 2053 del codice civile, della rovina di un edificio?", "Answer 1": "Il proprietario", "Answer 2": "Il fondo sociale per la protezione rischi", "Answer 3": "Il proprietario in solido con il conduttore, se presente"}
d9 = {"Question": "Ai sensi dell'art 97 della Costituzione, che cosa assicurano le pubbliche amministrazioni?", "Answer 1": " L'equilibrio dei bilanci e la sostenibilità del debito pubblico in coerenza con l'ordinamento dell'Unione europea ", "Answer 2": "Il rispetto del principio di trasparenza, correttezza e lealtà", "Answer 3": "Esclusivamente il pareggio di bilancio"}
d10 = {"Question": "Ai fini della tutela dell'assicurazione obbligatoria contro gli infortuni sul lavoro e le malattie professionali, l'indennizzo per il ristoro del danno biologico è erogato, ai sensi dell'art. 13 del D.Lgs. 38/2000:", "Answer 1": "In capitale e in rendita, in relazione al grado della menomazione.", "Answer 2": "In capitale e in rendita, in relazione all'età dell'infortunato.", "Answer 3": "In capitale e in rendita, indipendentemente dal grado della menomazione."}
d11 = {"Question": "Ai sensi dell'art. 73 della Costituzione, le leggi sono promulgate dal Presidente della Repubblica...:", "Answer 1": "entro un mese dall'approvazione", "Answer 2": "entro quindici giorni dall'approvazione", "Answer 3": "entro quaranta giorni dall'approvazione"}
d12 = {"Question": "In base all'art. 1577 del codice civile, il conduttore deve", "Answer 1": "dare avviso al locatore della necessità di eseguire quelle riparazioni alla cosa locata che non siano a proprio carico", "Answer 2": "corrispondere al locatore un'indennità per i miglioramenti apportati alla cosa locata senza il preventivo consenso del locatore stesso", "Answer 3": "garantire il locatore dalle molestie di terzi che pretendono di avere diritti sulla cosa locata"}
d13 = {"Question": "Ai sensi dell'art. 70 della Costituzione, da chi è esercitata la funzione legislativa?", "Answer 1": "è esercitata collettivamente dalle due Camere", "Answer 2": "è esercitata dal Presidente del Consiglio", "Answer 3": "è esercitata solo dalla camera del Senato"}
d14 = {"Question": "Secondo l'art. 283 c.p.c., il giudice d'appello può sospendere, in tutto o in parte, l'efficacia esecutiva della sentenza impugnata?", "Answer 1": "Sì, su istanza di parte, quando sussistano gravi e fondati motivi, anche in relazione alla possibilità di insolvenza di una delle parti.", "Answer 2": "Si, sempre d'ufficio", "Answer 3": "Sì, sempre d'ufficio, quando sussistano gravi e fondati motivi, anche in relazione alla possibilità di insolvenza di una delle parti."}
d15 = {"Question": "Quali sono, ai sensi dell'art. 85 c.p., i presupposti della imputabilità?", "Answer 1": "Capacità di intendere e volere al momento della commissione del fatto.", "Answer 2": "Capacità di intendere e di volere al momento della esecuzione della pena.", "Answer 3": "Capacità di agire"}
d16 = {"Question": "Secondo l'art. 2 comma 4 della L. 241/90 e ss.mm.ii., nel caso in cui siano indispensabili termini superiori a novanta giorni per la conclusione dei procedimenti di acquisto della cittadinanza italiana e di quelli riguardanti l'immigrazione, si possono superare i termini massimi, previsti dalla legge, dei 180 giorni?", "Answer 1": "Si", "Answer 2": "No, devono rientrare nei 180 giorni", "Answer 3": "Si, ma solo per i procedimenti riguardanti l'immigrazione"}
d17 = {"Question": "Ai sensi dell'art. 91 c.p., quali requisiti deve avere l'ubriachezza per escludere l'imputabilità del soggetto agente?", "Answer 1": "Deve derivare da caso fortuito o forza maggiore.", "Answer 2": "Non deve essere dolosa o preordinata alla realizzazione di un reato.", "Answer 3": "Deve essere colposa o accidentale."}
d18 = {"Question": "Ai sensi dell'art. 8, comma 2, del D.Lgs. 198/2006, il Comitato nazionale per l'attuazione dei principi di parità di trattamento e uguaglianza di opportunità tra lavoratori e lavoratrici è presieduto:", "Answer 1": "Dal Ministro del Lavoro o, per sua delega, da un Sottosegretario di Stato.", "Answer 2": "Dal Ministro dell'Economia e delle Finanze.", "Answer 3": "Da un Sottosegretario di Stato delegato dal Ministro del Lavoro o dal Segretario Generale."}
d19 = {"Question": "Quando il giudice può escludere la ripetizione delle spese sostenute dalla parte vincitrice (art. 92 c.p.c.)?", "Answer 1": "Quando le ritiene eccessive o superflue.", "Answer 2": "Mai, la compensazione dipende sempre dalla soccombenza.", "Answer 3": "Solo quando le parti si sono conciliate."}
d20 = {"Question": "In sede di esecuzione civile, se viene aggiudicato un bene gravato da pegno, il debitore è liberato (art. 508 c.p.c.)?", "Answer 1": "Sì, se il creditore pignoratizio consente e il giudice autorizza.", "Answer 2": "Sì, essendo sufficiente il consenso del creditore pignoratizio.", "Answer 3": "No, mai"}
d21 = {"Question": "Secondo l'art. 644 c.p.c., il decreto ingiuntivo diventa inefficace qualora non sia notificato nel termine di:", "Answer 1": "Sessanta giorni dalla pronuncia se la notifica deve avvenire nel territorio della Repubblica.", "Answer 2": "Centoventi giorni dalla pronuncia se la notifica deve avvenire in territorio estero.", "Answer 3": "Sessanta giorni dalla pronuncia se la notifica deve avvenire in territorio estero."}
d22 = {"Question": "In base all'art. 181 c.p.c., se l'attore costituito non compare alla prima udienza:", "Answer 1": "Il convenuto può chiedere che si proceda in assenza di lui.", "Answer 2": "Il convenuto non può chiedere che si proceda in assenza di lui.", "Answer 3": "Si procede d'ufficio in assenza di lui, se il convenuto è comparso."}
d23 = {"Question": "Quali sono le condotte tipiche alternativamente richieste dall'art. 340 c.p., per il reato di interruzione di ufficio o servizio pubblico o di pubblica necessità?", "Answer 1": "L'interruzione dell'ufficio o servizio pubblico o di un servizio di pubblica utilità e il turbamento della sua regolarità.", "Answer 2": "Il turbamento del servizio o dell'ufficio e la violenza verso il soggetto ad esso incaricato.", "Answer 3": "L'articolo 340 non indica condotte tipiche."}
d24 = {"Question": "In base all'art. 152 c.p.c., i termini perentori:", "Answer 1": "Non possono essere abbreviati o prorogati, nemmeno su accordo delle parti.", "Answer 2": "Non possono essere abbreviati, ma possono sempre essere prorogati su accordo delle parti.", "Answer 3": "Possono essere sempre e solo abbreviati dal giudice, su richiesta congiunta delle parti."}
d25 = {"Question": "Ai sensi dell'art. 2036 del codice civile, chi ha ricevuto l'indebito in mala fede", "Answer 1": "è tenuto a restituire i frutti e gli interessi", "Answer 2": "non è tenuto a restituire i frutti ne gli interessi", "Answer 3": "è tenuto a restituire i frutti ma non gli interessi"}
d26 = {"Question": "Ai sensi dell'art. 87 della Costituzione, chi Promulga le leggi ed emana i decreti aventi valore di legge e i regolamenti?", "Answer 1": "Il presidente della repubblica", "Answer 2": "Il presidente del consiglio", "Answer 3": "Il ministro di grazia e giustizia"}
d27 = {"Question": "Quando, ai fini dell'efficacia di un titolo esecutivo, occorre prestare cauzione, la sua prestazione si può far constare con (art. 478 c.p.c.):", "Answer 1": "Atto separato, unito al titolo.", "Answer 2": "Qualsiasi forma idonea.", "Answer 3": "Atto separato, notificato anteriormente al titolo."}

d28 = {"Question": "Qual è la conseguenza della mancata costituzione in giudizio del terzo il cui intervento sia stato ordinato dal giudice ai sensi dell’art. 108 c.p.c., qualora una delle parti abbia ritualmente provveduto alla sua chiamata in causa?", "Answer 1": "Il giudice dichiara la contumacia del terzo", "Answer 2": "Il giudice dispone la cancellazione della causa dal ruolo", "Answer 3": "Il giudice ordina l’accompagnamento coattivo in udienza del terzo"}
d29 = {"Question": "Quale mezzo di impugnazione è ammissibile avverso l’ordinanza di sospensione necessaria del processo ex art. 295 c.p.c.?", "Answer 1": "Il regolamento necessario di competenza", "Answer 2": "Il ricorso per cassazione", "Answer 3": "L’appello"}
d30 = {"Question": "Il ricorso in appello deve essere notificato a pena di inammissibilità entro il termine previsto dall’art. 92 c.p.a.:", "Answer 1": "ad almeno una delle parti interessate a contraddire", "Answer 2": "alle parti interessate a contraddire", "Answer 3": "alle sole parti costituite nel giudizio di primo grado"}
d31 = {"Question": "Nel giudizio avverso il silenzio di cui agli artt. 31 e 117 c.p.a. il Giudice può pronunciare sulla fondatezza della pretesa dedotta in giudizio:", "Answer 1": "solo quando si tratti di attività vincolata o quando risulta che non residuano ulteriori margini di esercizio della discrezionalità e non sono necessari adempimenti istruttori che debbano essere compiuti dall’amministrazione", "Answer 2": "solo quando non siano necessari adempimenti istruttori che debbano essere compiuti dall’amministrazione", "Answer 3": "solo quando non residuino ulteriori margini di esercizio della discrezionalità"}

df_simsim = pd.DataFrame([d, d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14, d15, d16, d17, d18, d19, d20, d21, d22, d23, d24, d25, d26, d27, d28, d29, d30, d31])
df_simsim.shape

(32, 4)

# Merging All Dataframes

In [23]:
# Merge the two dataframes and clean up the data
df_merged = pd.concat([df_diramm, df_domjson, df_inps, df_minin, df_forgiur, df_simsim], ignore_index=True)
df_merged.to_csv(DEFAULT_SAVE_DIR + 'quiz_merged_orig.csv', index=False)

# Delete rows which don't contain a number (a law reference)
df_merged = df_merged[df_merged['Question'].apply(lambda x: bool(re.search(r'\d', x)))]
df_merged = df_merged.drop_duplicates(subset='Question')

# For each column, convert multiple spaces and \n to single spaces
for col in df_merged.columns:
    df_merged[col] = df_merged[col].apply(lambda x: re.sub(r'\s\s+|\n', ' ', str(x)))

df_merged.insert(0, 'Index', range(1, 1 + len(df_merged)))

df_merged.to_csv(DEFAULT_SAVE_DIR + 'quiz_merged.csv', index=False)
print(df_merged.shape)
df_merged.head()

(1093, 5)


,Index,Question,Answer 1,Answer 2,Answer 3
1,1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
3,2,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,3,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...
7,4,A norma di quanto dispone l'art. 133 del C.p.a...,Alla giurisdizione esclusiva del giudice ammin...,Alla giurisdizione esclusiva del giudice ordin...,Alla giurisdizione esclusiva del TAR del Lazio.
8,5,Consacrando a livello costituzionale i princip...,"Moneta, tutela del risparmio e mercati finanzi...",Tutela e sicurezza del lavoro.,"Produzione, trasporto e distribuzione nazional..."
